#  信用卡逾期二元分类

# 课程目标

* 创建模型：如何创建模型、分层交叉验证和评估分类结果

* 调整模型：如何自动调整分类模型的超参数

* 绘图模型：如何使用各种绘图分析模型性能

* 完成模型：如何在实验结束时确定最佳模型

* 预测模型：如何对新的/未看到的数据进行预测

* 保存/加载模型：如何保存/加载模型以备将来使用


## 什么是二元分类
二元分类是一种有监督的机器学习技术，其目标是预测离散且未标注的分类标签，如通过/失败、正/负、默认/非默认等。下面列出了几个实际分类用例：

- 用于确定患者是否患有某种疾病的医学测试——分类属性是该疾病的是否存在。

- 工厂中的“合格或不合格”测试方法或质量控制，即确定是否符合规范——分类属性是通过/不通过分类。

- 信息检索，即决定一个页面或一篇文章是否应该出现在搜索结果集中——分类属性是文章的相关性，或者对用户的有用性。

## PyCaret中的分类模块概述
PyCaret分类模块(`pycaret.classification`)是一个有监督的机器学习模块，用于根据各种技术和算法对目标进行二元分类。分类问题的一些常见用例包括预测客户违约（是或否）、客户流失（客户将离开或留下）、发现疾病（阳性或阴性）。
PyCaret分类模块可以用于二元分类或多元分类情况。它有超过18个算法和14种图来分析模型的性能。包括对于是超参数调整、堆叠等高级技术，PyCaret的分类模块均能应对自如。

# 二元分类初探
##  数据准备
### 二元分类实例数据集

在本教程中，我们将使用UCI的一个数据集，称为**Default of Credit Card Clients Dataset**。数据集包含2005年4月至2005年9月台湾信用卡客户的拖欠付款、人口统计因素、信用资料、付款历史及帐单结算单等资料。有30000个样本和25个特征：

- **ID:** 每个客户端的ID
- **LIMIT_BAL:** 限额余额：以新台币计算之信用额度（含个人及家庭/补充信用）
- **SEX:** 性别（1=男性，2=女性）
- **EDUCATION:** 学历（1=研究生院，2=大学，3=高中，4=其他，5=未知，6=未知）
- **MARRIAGE:** 婚姻状况（1=已婚，2=单身，3=其他）
- **AGE:** 以年为单位的年龄
- **PAY_0 to PAY_6:** n个月前的还款状态（PAY_0=上个月……PAY_6=6个月前）（标签：-1=按时付款，1=延迟一个月付款，2=延迟两个月付款……8=延迟付款8个月，9=延迟付款9个月及以上）
- **BILL_AMT1 to BILL_AMT6:** n个月前的账单金额（BILL_AMT1=上个月……BILL_AMT6=6个月前）
- **PAY_AMT1 to PAY_AMT6:** n个月前的付款金额（BILL_AMT1=上个月……BILL_AMT6=6个月前）
- **default:** 默认分类目标，表征是否会付款（1=是，0=否）

**数据集信息** 
Lichman, M. (2013). UCI Machine Learning Repository. Irvine, CA: University of California, School of Information and Computer Science. 

### 获取数据集

或者，可以使用PyCaret的数据存储库，使用“get_data()”加载数据功能（可能需要梯子）。

In [ ]:
# 通过pycaret加载
from pycaret.datasets import get_data

In [ ]:
# pycaret 自带数据集
all_datasets = get_data('index')

In [ ]:
dataset = get_data('credit')

如果无法联网，从__[原始源](https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients)__下载数据 ，将下载后的数据放在/data 目录下，并用pandas完成加载。

In [ ]:
# import pandas as pd
# file_name = './data/credit_card.csv'
# dataset = pd.read_csv(file_name)

In [ ]:
# 检查数据，亦可使用profile查看数据大体分布
dataset.shape

**对数据做轮廓分析**

In [ ]:
import pandas_profiling
pfr = pandas_profiling.ProfileReport(dataset)
pfr.to_file('report.html')
pfr

### 划分数据验证集Validation Set
为了演示“predict_model()”函数对未知的数据的作用，从原始数据集中截取了1500条样本用于最终验证测试。这不应与训练/测试集的拆分相混淆，因为执行此特定拆分是为了模拟真实场景。另一种思考方法是，在进行机器学习实验时，这1500条记录是不可用的，模型无法见到。

In [ ]:
# 划分训练集，random_state是随机数种子
data = dataset.sample(frac=0.95, random_state=786)
data_unseen = dataset.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

### PyCaret初始化准备
“setup()”函数的作用是初始化pycaret中的环境，并创建转换管道，为建模和部署准备数据。`必须在执行pycaret中的任何其他函数之前调用setup()`。它需要两个必需的参数：pandas dataframe和目标列的名称。所有其他参数都是可选的，用于自定义预处理管道（我们将在后面的课程中看到它们）。

当执行“setup()”时，PyCaret的推理算法将根据某些属性自动推断所有特性的数据类型。使用应当正确判断每一列的数据类型，但情况并非总是如此。为了说明这一点，PyCaret在执行“setup()”之后显示一个包含特性及其推断数据类型的表。如果所有数据类型都正确识别，可按“回车”继续，或键入“quit”结束实验。确保数据类型是正确的在PyCaret中是至关重要的，因为它自动执行一些预处理任务，这些任务对于任何机器学习实验都是必不可少的。对于每种数据类型，执行这些任务的方式不同，这意味着正确的配置非常重要。

In [ ]:
from pycaret.classification import *

In [ ]:
# 运行过程中输入Enter进行确认
exp_clf= setup(data = data, target = 'default', session_id=123) 

一旦成功执行，它将打印包含几个重要信息的信息表格。大部分信息与执行“setup()”时构造的预处理管道有关。这些功能中的大多数都超出了本教程的范围，但是在此阶段需要注意的一些重要事项包括：

- **session_id :**  一个随机数，作为随机种子分布在所有函数中，以便以后的代码运行能够再现。如果没有传递“session_id”，则会自动生成一个随机数，并分发给所有函数。在本实验中，“session_id”被设置为“123”，以便以后重现。<br/>
<br/>
- **Target Type :**  二元分类或多元分类。自动检测并显示目标类型。对于二元分类或多元分类问题，实验的执行方式没有区别。所有功能都相同。<br/>
<br/>
- **Label Encoded :**  当目标变量的类型为string（即“Yes”或“No”）而不是1或0时，它会自动将标签编码为1和0，并显示映射（0:No，1:Yes）以供参考。在这个实验中，不需要标签编码，因为目标变量的类型是numeric。<br/>
<br/>
- **Original Data :**  显示数据集的原始形状。 <br/>
<br/>
- **Missing Values :** 如果原始数据中缺少值，则显示为True。对于这个实验，数据集中没有丢失的值。<br/>
<br/>
- **Numeric Features :**  推断为数字的特征数。在这个数据集中，25个特征中有15个被推断为数值特征。 <br/>
<br/>
- **Categorical Features :**  推断为类别特征的数量。在这个数据集中，25个特征中有9个被推断为分类特征。 <br/>
<br/>
- **Transformed Train Set :**  显示转换后训练集的形状。请注意，对于转换后的列车组，原始形状（28500, 25）被转换为（19949, 92），由于分类编码，特征的数量从25增加到92<br/>
<br/>
- **Transformed Test Set :**  显示转换后的测试集的形状。测试数据集中有8551个样本。总样本在70/30参数的基础上进行训练集与测试集的划分，可以使用‘train_size’参数的默认值来更改划分比例。<br/>

请注意，执行建模所必需的一些任务是如何自动处理的，例如缺失值插补（在本例中，训练数据中没有缺失值，但我们仍需要未查看数据的插补器）、分类编码等。“setup()”中的大多数参数都是可选的，用于定制预处理管道。

## 模型比较

在设置完成后，建议从比较所有模型以评估性能为出发点（除非您确切知道您需要什么样的模型，而事实往往并非如此）。该函数训练模型库中的所有模型，并使用分层交叉验证对其进行度量评估。输出将打印一个分数表格，显示每种分类器的平均准确度、AUC、召回率、精确率、F1分数、Kappa和MCC以及训练时间。

对于元二分类问题：
* 真正例（TP）：实际上是正例的数据点被标记为正例
* 假正例（FP）：实际上是反例的数据点被标记为正例
* 真反例（TN）：实际上是反例的数据点被标记为反例
* 假反例（FN）：实际上是正例的数据点被标记为反例

评价指标：
* Accuracy: 准确率=(TP+TN)/(TP+TN+FP+FN)
* AUC (Area Under the Curve)：结合2.5部分进行理解
* Recall：召回率=TP/(TP+FN)——模型对于正例的识别准确度
* Prec.（Precision）：精确率=TP/(TP+FP)——所有返回结果为正例的样本中，真正为正例的比例
* F1（F1-score）：精确率和召回率的调和平均数=`2*Prec.*Recall/(Prec.+Recall)`
* Kappa——Kappa系数，衡量预测一致性的__[一种指标](https://baike.baidu.com/item/kappa%E7%B3%BB%E6%95%B0/9385025?fr=aladdin)__
* MCC（Matthews Correlation coefficient）——马修斯相关性系数，适用于评估样本分布不平衡的数据集
* TT：训练时间

以上指标除训练时间外均越大越好，取值范围均为[0,1]
一份__[整体说明](https://www.jianshu.com/p/c61ae11cc5f6)__

In [ ]:
best_model = compare_models()

默认情况下，网格使用`Accuracy`准确率（从高到低）排序，通过传递“排序”参数可以更改该网格。例如，`compare_models(sort = 'Recall')`将按召回率对网格进行排序。如果要将折叠参数从默认值“10”更改为其他值，则可以使用“折叠”参数，例如`compare_models(fold = 5)`。默认情况下，`compare_models`根据默认排序顺序返回最佳性能模型，但可以使用 `n_select` 参数返回最好的N个模型列表。

In [ ]:
print(best_model)

## 制作模型

`create_model` 是PyCaret中最细粒度的函数，通常是大多数PyCaret函数背后的基础。正如名称所示，此函数能够使用`fold`参数来设置K-折交叉验证从而用于训练和评估模型。输出打印一个分数表格，显示准确率、AUC、召回率、精确率、F1分数、Kappa和MCC。

在本实例的剩余部分，我们将使用以下模型作为候选模型。这些选择仅用于说明目的，并不一定意味着它们是最优秀的或理想的此类数据。

- 决策树分类器（'dt'）
- K邻域分类器（'knn'）
- 随机森林分类器（'rf'）

PyCaret模型库中有18个分类器可用。要查看所有分类器列表，请检查`docstring`或使用`models`函数查看库。

In [ ]:
models()

### 决策树分类器

In [ ]:
dt = create_model('dt')

In [ ]:
# 所训练的模型对象保存在变量'dt'中
print(dt)

### KNN分类器

In [ ]:
knn = create_model('knn')

### 随机森林分类器

In [ ]:
rf = create_model('rf')

请注意，所有模型的平均分数都与`compare_models()`中打印的分数相匹配。这是因为`compare_models()`分数网格中打印的指标是所有K-Fold交叉验证的平均分数。与`compare_models()`类似，如果要将fold参数从默认值10更改为其他值，则可以使用`fold`参数。例如：`create_model('dt', fold = 5)`将使用5折交叉验证创建一个决策树分类器。

# 模型优化

当使用`create_model()`函数创建模型时，它将使用默认的超参数来训练模型。为了优化超参数，使用了`tune_model()`函数。此函数在预定义的搜索空间中使用`Random Grid Search` 自动调整模型的超参数。输出将打印一个分数网格，按Fold显示最佳模型的准确率、AUC、召回率、精确率、F1分数、Kappa和MCC。要使用自定义搜索网格，可以在`tune_model()`函数中传递`custom_grid`参数（请参见下面的KNN tuning）。<br/>
<br/>

### 优化决策树分类器

In [ ]:
tuned_dt = tune_model(dt)

In [ ]:
# 优化后的模型对象储存在'tuned_dt'变量中. 
print(tuned_dt)

### KNN 分类器

In [ ]:
import numpy as np
tuned_knn = tune_model(knn, custom_grid = {'n_neighbors' : np.arange(0,50,1)})

In [ ]:
print(tuned_knn)

### 随机森林分类器

In [ ]:
tuned_rf = tune_model(rf)
print(tuned_rf)

默认情况下，`tune_model`优化`Accuracy`准确率指标，但可以使用`optimize`参数更改此设置。例如： `tune_model(dt, optimize = 'AUC')`将搜索决策树分类器的超参数，该分类器的结果是`AUC`参数最高的模型，而不是`Accuracy`。对于该实例，我们使用了默认的度量`Accuracy`，只是为了简单起见。一般来说，当数据集不平衡时（例如我们正在处理的信贷数据集），`Accuracy`不是一个很好的衡量标准。选择正确的度量来评估分类器的方法超出了本教程的范围，但是如果想了解更多信息，可以__[click here](https://medium.com/@MohammedS/performance-metrics-for-classification-problems-in-machine-learning-part-i-b085d432082b)__ 阅读一篇关于如何选择正确评估指标的文章。

在最终确定最佳模型时，单一指标并不是应该考虑的唯一标准。其他需要考虑的因素包括训练时间、k-folds的标准偏差等。现在，假设我们选择了随机森林分类器“tuned_rf”。


## 模型可视化

在模型定型之前，`plot_model()` 函数可用于分析不同方面的性能，如混淆矩阵、ROC曲线图、P-R曲线图等。此函数接受一个经过训练的模型对象，并根据测试集返回绘图。

有15种不同的绘图，请参见`plot_model()` 文档获取可用绘图的列表。

### 2.5.1 混淆矩阵

给定一个模型的预测标签时，它可以被用来快速计算精度和召回率。
二元分类的混淆矩阵总共包含四个不同的结果：真正例（TP）、假正例（FP）、真反例（TN），以及假反例（FN）。纵坐标代表真实标签，横坐标代表预测标签。
图中左上角代表TN，右上角代表FP，左下角代表FN，右下角代表TP，相关指标：

* Accuracy: 准确率=(TP+TN)/(TP+TN+FP+FN)
* Recall：召回率=TP/(TP+FN)——模型对于正例的识别准确度
* Prec.（Precision）：精确率=TP/(TP+FP)——所有返回结果为正例的样本中，真正为正例的比例
* F1（F1-score）：精确率和召回率的调和平均数=`2*Prec.*Recall/(Prec.+Recall)`

计算结果：
TP=641,TN=6377,FN=1254,FP=279
* Accuracy:(TP+TN)/(TP+TN+FP+FN)=0.8207
* Recall：TP/(TP+FN)=0.3383
* Prec.（Precision）：TP/(TP+FP)=0.6967
* F1（F1-score）：`2*Prec.*Recall/(Prec.+Recall)`=0.4554


In [ ]:
plot_model(tuned_rf, plot = 'confusion_matrix')

### ROC曲线与AUC值

ROC（receiver operating characteristic），平面的横坐标是false positive rate(FPR)假阳率，表征纵坐标是true positive rate(TPR)真阳率。

* FPR=FP/(FP+TN)
* TPR=TP/(TP+FN)——意同召回率Recall

ROC曲线绘制方式：

- 1.假设已经得出一系列样本被划分为正类的概率Score值，按照大小排序。
- 2.从高到低，依次将“Score”值作为阈值threshold，当测试样本属于正样本的概率大于或等于这个threshold时，我们认为它为正样本，否则为负样本。举例来说，对于某个样本，其“Score”值为0.6，那么“Score”值大于等于0.6的样本都被认为是正样本，而其他样本则都认为是负样本。
- 3.每次选取一个不同的threshold，得到一组FPR和TPR，以FPR值为横坐标和TPR值为纵坐标，即ROC曲线上的一点。
- 4.根据3中的每个坐标点，画图。

#### AUC值

AUC表示ROC曲线下的面积，主要用于衡量模型的泛化性能，即分类效果的好坏。AUC是衡量二分类模型优劣的一种评价指标，表示正例排在负例前面的概率。一般在分类模型中，预测结果都是以概率的形式表现，如果要计算准确率，通常都会手动设置一个阈值来将对应的概率转化成类别，这个阈值也就很大程度上影响了模型准确率的计算。

之所以采用AUC来评价，主要还是考虑到ROC曲线本身并不能直观的说明一个分类器性能的好坏，而AUC值作为一个数量值，具有可比较性，可以进行定量的比较。

In [ ]:
plot_model(tuned_rf, plot = 'auc')

### P-R曲线

P-R曲线即为精确召回率曲线，类似ROC曲线的绘制，PR曲线的每个点要对应一个阈值。通过选择合适的阈值，比如50%，对样本进行划分，概率大于50%的就认为是正例，小于50%的就是负例,从而计算相应的精准率和召回率，并最终连线作图。

In [ ]:
plot_model(tuned_rf, plot = 'pr')

### 特征重要性图

特征对目标变量预测的相对重要性，可以通过决策树中使用特征作为决策节点的相对顺序来评估。
决策树顶部使用的特征，将对更多样本的最终预测决策做出贡献。
因此，可以通过每个特征对最终预测做出贡献的样本比例，来评估该特征的重要性。

In [ ]:
plot_model(tuned_rf, plot='feature')

### PyCaret所包含的图表展示
另一种分析模型性能的方法是使用`evaluate_model()` 函数，该函数为给定模型的所有可用绘图显示用户界面。它在内部使用 `plot_model()` 函数。如

* Hyperparameters列出了该分类器的所有超参列表
* Validation Curve表征随训练准确度上升，交叉验证得分的变化趋势
* Threshold表征随选择的阈值threshold不同，各项指标的变化趋势

In [ ]:
evaluate_model(tuned_rf)

## 在测试集上进行预测

在最终确定模型之前，建议通过预测测试集和评审评估指标来执行一次最终检查。如果查看上面第2.1.4节中的信息表格，将看到30%的数据被分离为测试样本。我们在上面看到的所有评估指标都是基于训练集（70%）的交叉验证结果。现在，使用存储在“tuned_rf”变量中的最终训练模型，我们将根据保留样本进行预测，并评估指标，以确定它们是否与交叉验证的结果有实质性差异。

In [ ]:
predict_model(tuned_rf)

测试集的准确度为**`0.8207`** ，而`tuned_rf`交叉验证结果的准确度为**`0.8169`** （见上文第2.4.3节）。这并不是一个显著的区别。如果测试集和交叉验证结果之间存在较大差异，则通常表明模型存在过度拟合，但也可能是由于其他几个因素造成的，需要进一步调查。在本例中，我们将继续完成模型并对未看到的数据（我们在开始时分离的5%的数据从未公开给PyCaret）进行预测。

（提示：使用`create_model()`时，最好查看交叉验证结果的标准差。）


## 最终模型部署

模型定型是实验的最后一步。PyCaret中一个正常的机器学习工作流程从 `setup()`开始，然后使用 `compare_models()` 比较所有模型，并列出一些候选模型（基于感兴趣的度量），以执行一些建模技术，如超参数调整、集成和堆叠等。这个工作流程最终将引导我们找到最佳模型，用于对新的和未看到的数据进行预测。`finalize_model()`函数将模型放入完整的数据集，包括测试样本（在本例中为30%）。此函数的目的是在将模型部署到生产环境中之前，在完整的数据集上训练模型。

In [ ]:
final_rf = finalize_model(tuned_rf)

In [ ]:
# Final Random Forest model parameters for deployment
print(final_rf)

**Caution:** 使用 `finalize_model()`完成模型后，整个数据集（包括测试集）将用于训练。因此，如果在使用 `finalize_model()`之后使用模型对保留集进行预测，则打印的信息表格将具有误导性，因为试图对用于建模的相同数据进行预测。为了证明这一点，我们将使用 `predict_model()`下的 `final_rf` ，将信息网格与上面网格进行比较。

In [ ]:
predict_model(final_rf);

请注意，即使模型是相同的,`final_rf`中的AUC从**`0.7727`** 增加到**`0.8275`** 。这是因为`final_rf`变量已经在包括测试集在内的完整数据集上进行了训练。

## 在验证集上进行模型验证

 `predict_model()` 函数也用于预测未进行预测的验证数据集。与上面2.6的唯一区别是，这次我们将传递`data_unseen` 参数。`data_unseen` 是在教程开始时创建的变量，包含5%（1500个示例）的原始数据集，该数据集从未公开给PyCaret。

In [ ]:
unseen_predictions = predict_model(final_rf, data=data_unseen)
unseen_predictions.head()

Label和Score列被添加到 `data_unseen`集中。Label是预测，score是预测的概率。请注意，当所有的转换都在后台自动执行时，预测的结果将连接到原始数据集。您还可以检查这个指标，因为您有实际的目标列default可用。为此，我们将使用`pycaret.utils`模块。参见以下示例，最终我们模型在验证数据集1500份上的准确率为81.67%

In [ ]:
from pycaret.utils import check_metric
check_metric(unseen_predictions.default, unseen_predictions.Label.astype('int'), 'Accuracy')

## 保存模型

我们现在完成了实验，最终完成了`tuned_rf`模型，该模型现在存储在 `final_rf`变量中。我们还使用了存储在 `final_rf`中的模型来预测`data_unseen`。我们的实验到此结束，但还有一个问题需要问：当有更多的新数据要预测时会发生什么？你必须把整个实验再看一遍吗？答案是否定的，PyCaret的内置函数 `save_model()`允许我们将模型与整个转换管道一起保存，以供以后使用。

In [ ]:
save_model(final_rf,'Final RF Model 14Oct2021')

（提示：保存模型时最好在文件名中使用日期，这有利于版本控制。）

## 读取模型

要在将来的某个日期在同一环境或其他环境中加载保存的模型，我们将使用PyCaret的`load_model()`函数，然后轻松地将保存的模型应用于新的未查看数据上以进行预测。

In [ ]:
saved_final_rf = load_model('Final RF Model 14Oct2021')

一旦模型加载到环境中，就可以使用相同的`predict_model()`函数来预测任何新数据。下面我们应用加载模型来预测我们在上面使用的相同的 `data_unseen` 

In [ ]:
new_prediction = predict_model(saved_final_rf, data=data_unseen)

In [ ]:
new_prediction.head()

注意到`unseen_predictions` 与 `new_prediction`的预测结果准确率是相同的

In [ ]:
from pycaret.utils import check_metric
check_metric(new_prediction.default, new_prediction.Label.astype('int'), 'Accuracy')